# Chart generations Notebook

## Import Libraries

In [1]:
import pandas as pd
import plotly.graph_objs as go
import plotly.offline as offline
import os
import numpy as np
from plotly.offline import plot
from pathlib import Path

In [2]:
offline.init_notebook_mode(connected=True)

## Bubble Chart for Emissions Intensity


In [3]:
company_id = 66

# Step 1: Set up the data sources

In [4]:
path = Path(os.path.dirname (os.getcwd()))
XLSX_PATH = os.path.join(path.parent.parent, 'sp100.xlsx')

COLS_TO_USE = {
    'companies':['company_id', 'company_name'],
    'ghg_quant':['company_id', 'ghg_scope_1','ghg_loc_scope_2','ghg_mkt_scope_2', 'reporting_year', "Source"],
    'grouping':['company_id', 'Sector1'],
    'financials':['company_id', 'Revenue_num_tradingview']
}


# Step 2: Merge the tables

In [5]:
all_df = []
for sheetname, cols in COLS_TO_USE.items():
    all_df.append(pd.read_excel(
        XLSX_PATH, 
        sheet_name = sheetname,
        engine = 'openpyxl', 
        usecols = cols
        ))

In [6]:
merged_df = all_df[0]
for i in range(len(COLS_TO_USE) - 1):
    merged_df = pd.merge(
        left = merged_df,
        right = all_df[i+1],
        how="left",
        on="company_id"
    )

In [7]:
merged_df

,company_id,company_name,Source,reporting_year,ghg_scope_1,ghg_loc_scope_2,ghg_mkt_scope_2,Sector1,Revenue_num_tradingview
0,1.0,3M,CDP,2020.0,NR,NR,NR,Industrials,3.218400e+10
1,1.0,3M,Public,2020.0,3600000,1680000,1250000,Industrials,3.218400e+10
2,1.0,3M,Final,2020.0,3600000,1680000,1250000,Industrials,3.218400e+10
3,1.0,3M,CDP,2019.0,4050000,1780000,1320000,Industrials,3.218400e+10
4,1.0,3M,Public,2019.0,4050000,1780000,1320000,Industrials,3.218400e+10
...,...,...,...,...,...,...,...,...,...
1351,113.0,Linde plc,Public,2018.0,16872000,NR,22333000,NaN,2.724300e+10
1352,113.0,Linde plc,Final,2018.0,16872000,0,22333000,NaN,2.724300e+10
1353,113.0,Linde plc,CDP,2017.0,NR,NR,NR,NaN,2.724300e+10
1354,113.0,Linde plc,Public,2017.0,NR,NR,NR,NaN,2.724300e+10


In [8]:
# Select Final Figures and Year 2019
cond1 = (merged_df['reporting_year'] == 2019)
cond2 = (merged_df['Source']== 'Final')
filter_cond = cond1 & cond2
merged_df = merged_df.loc[filter_cond]

In [9]:
merged_df

,company_id,company_name,Source,reporting_year,ghg_scope_1,ghg_loc_scope_2,ghg_mkt_scope_2,Sector1,Revenue_num_tradingview
5,1.0,3M,Final,2019.0,4050000,1780000,1320000,Industrials,3.218400e+10
17,2.0,Abbott Laboratories,Final,2019.0,533000,518000,439000,Healthcare,3.460800e+10
29,3.0,AbbVie Inc.,Final,2019.0,314421,308204,249777,Healthcare,4.580400e+10
41,4.0,Accenture,Final,2019.0,18923,281489,214680,Technology,4.432700e+10
53,5.0,Adobe Inc.,Final,2019.0,11816,56113,43526,Technology,1.286500e+10
...,...,...,...,...,...,...,...,...,...
1301,109.0,Halliburton Co.,Final,2019.0,4511499,249259,NR,NaN,NaN
1313,110.0,HP Inc (ex Hewlett-Packard),Final,2019.0,57509,313002,177018,NaN,NaN
1325,111.0,Norfolk Southern Corp.,Final,2019.0,4784047,201474,201474,NaN,NaN
1337,112.0,T-Mobile (ex T-Mobile US),Final,2019.0,37175,1393730,729490,NaN,6.839700e+10


In [10]:
merged_df.head()

,company_id,company_name,Source,reporting_year,ghg_scope_1,ghg_loc_scope_2,ghg_mkt_scope_2,Sector1,Revenue_num_tradingview
5,1.0,3M,Final,2019.0,4050000,1780000,1320000,Industrials,3.218400e+10
17,2.0,Abbott Laboratories,Final,2019.0,533000,518000,439000,Healthcare,3.460800e+10
29,3.0,AbbVie Inc.,Final,2019.0,314421,308204,249777,Healthcare,4.580400e+10
41,4.0,Accenture,Final,2019.0,18923,281489,214680,Technology,4.432700e+10
53,5.0,Adobe Inc.,Final,2019.0,11816,56113,43526,Technology,1.286500e+10


# Step 3: Filter the sector

In [11]:
a =  merged_df[merged_df['company_id']==company_id]['Sector1']
if len(a) > 0:
    sector = a.iloc[0]
merged_df_sector = merged_df.loc[merged_df['Sector1']==sector]

In [12]:
merged_df_sector.head()

,company_id,company_name,Source,reporting_year,ghg_scope_1,ghg_loc_scope_2,ghg_mkt_scope_2,Sector1,Revenue_num_tradingview
41,4.0,Accenture,Final,2019.0,18923,281489,214680,Technology,4.432700e+10
53,5.0,Adobe Inc.,Final,2019.0,11816,56113,43526,Technology,1.286500e+10
161,14.0,Apple Inc.,Final,2019.0,50549,862127,0,Technology,2.741500e+11
317,27.0,Cisco Systems,Final,2019.0,41181,651331,187428,Technology,4.981800e+10
629,53.0,IBM Corp.,Final,2019.0,80159,987066,827369,Technology,7.362000e+10


# Step 4: Calculate Scope 1 + Scope 2

In [13]:
scope1 = pd.to_numeric(merged_df_sector['ghg_scope_1'], errors = 'coerce')
scope2_loc = pd.to_numeric(merged_df_sector['ghg_loc_scope_2'], errors = 'coerce')
scope2_mkt = pd.to_numeric(merged_df_sector['ghg_mkt_scope_2'], errors = 'coerce')

merged_df_sector = merged_df_sector.assign(scope1_plus_scope2_loc= scope1 + scope2_loc)
merged_df_sector = merged_df_sector.assign(scope1_plus_scope2_mkt= scope1 + scope2_mkt)
merged_df_sector['scope1_plus_scope2'] = merged_df_sector[['scope1_plus_scope2_loc','scope1_plus_scope2_mkt']].min(axis=1)

In [14]:
fields = ['company_id','company_name','ghg_scope_1', 'Sector1','Revenue_num_tradingview','scope1_plus_scope2']
merged_df_sector = merged_df_sector [fields]

In [15]:
merged_df_sector

,company_id,company_name,ghg_scope_1,Sector1,Revenue_num_tradingview,scope1_plus_scope2
41,4.0,Accenture,18923,Technology,4.432700e+10,233603.0
53,5.0,Adobe Inc.,11816,Technology,1.286500e+10,55342.0
161,14.0,Apple Inc.,50549,Technology,2.741500e+11,50549.0
317,27.0,Cisco Systems,41181,Technology,4.981800e+10,228609.0
629,53.0,IBM Corp.,80159,Technology,7.362000e+10,907528.0
641,54.0,Intel Corp.,1580000,Technology,7.786700e+10,2879000.0
785,66.0,Microsoft Corp.,113412,Technology,1.680880e+11,388787.0
857,72.0,Nvidia,2817,Technology,1.667500e+10,68753.0
881,74.0,Oracle,16520,Technology,4.047900e+10,365542.0
953,80.0,QUALCOMM Inc.,78290,Technology,2.353100e+10,192350.0


# Step 5: Remove companies without valid data

In [16]:
merged_df_sector

,company_id,company_name,ghg_scope_1,Sector1,Revenue_num_tradingview,scope1_plus_scope2
41,4.0,Accenture,18923,Technology,4.432700e+10,233603.0
53,5.0,Adobe Inc.,11816,Technology,1.286500e+10,55342.0
161,14.0,Apple Inc.,50549,Technology,2.741500e+11,50549.0
317,27.0,Cisco Systems,41181,Technology,4.981800e+10,228609.0
629,53.0,IBM Corp.,80159,Technology,7.362000e+10,907528.0
641,54.0,Intel Corp.,1580000,Technology,7.786700e+10,2879000.0
785,66.0,Microsoft Corp.,113412,Technology,1.680880e+11,388787.0
857,72.0,Nvidia,2817,Technology,1.667500e+10,68753.0
881,74.0,Oracle,16520,Technology,4.047900e+10,365542.0
953,80.0,QUALCOMM Inc.,78290,Technology,2.353100e+10,192350.0


In [17]:
merged_df_sector = merged_df_sector.dropna()

# Step 6 (optional): Calculate GHG Intensity

In [18]:
merged_df_sector = merged_df_sector.assign(intensity=merged_df_sector['scope1_plus_scope2'] / merged_df_sector['Revenue_num_tradingview'])

In [19]:
merged_df_sector

,company_id,company_name,ghg_scope_1,Sector1,Revenue_num_tradingview,scope1_plus_scope2,intensity
41,4.0,Accenture,18923,Technology,4.432700e+10,233603.0,5.269993e-06
53,5.0,Adobe Inc.,11816,Technology,1.286500e+10,55342.0,4.301749e-06
161,14.0,Apple Inc.,50549,Technology,2.741500e+11,50549.0,1.843845e-07
317,27.0,Cisco Systems,41181,Technology,4.981800e+10,228609.0,4.588884e-06
629,53.0,IBM Corp.,80159,Technology,7.362000e+10,907528.0,1.232719e-05
641,54.0,Intel Corp.,1580000,Technology,7.786700e+10,2879000.0,3.697330e-05
785,66.0,Microsoft Corp.,113412,Technology,1.680880e+11,388787.0,2.312997e-06
857,72.0,Nvidia,2817,Technology,1.667500e+10,68753.0,4.123118e-06
881,74.0,Oracle,16520,Technology,4.047900e+10,365542.0,9.030411e-06
953,80.0,QUALCOMM Inc.,78290,Technology,2.353100e+10,192350.0,8.174323e-06


# Step 7: Prepare the chart by fetching specific data series

In [20]:
corp_x = merged_df_sector[merged_df_sector['company_id']==company_id]['scope1_plus_scope2'].iloc[0]
corp_y = merged_df_sector[merged_df_sector['company_id']==company_id]['Revenue_num_tradingview'].iloc[0]
corp_name = merged_df_sector[merged_df_sector['company_id']==company_id]['company_name'].iloc[0]
x0 = merged_df_sector['scope1_plus_scope2']
y0 = merged_df_sector['Revenue_num_tradingview']
intensity_data = 10000000 * merged_df_sector['intensity']
#index_corp = merged_df_sector.loc[merged_df_sector['company_id'] == company_id].index
#merged_df_sector.drop(index_corp , inplace=True)
x_median_x = [x0.min(), x0.max()]
y_median_x = [y0.median(), y0.median()]
x_median_y = [x0.median(), x0.median()]
y_median_y = [y0.min(), y0.max()]

# Step 7: Produce the chart

In [49]:
layout = go.Layout (
    title = '<b>Operational Emissions Intensity Benchmark</b><br>Sector: ' + sector,
    title_x = 0.5,
    titlefont = dict(
        family = 'Arial',
        size = 16),
    plot_bgcolor = 'antiquewhite',
    xaxis =  dict(
        autorange = "reversed",
        type = 'log',
        title = '<i>high</i>----------<b>Scope1+2 Emissions</b>----------<i>low</i>'),
    yaxis = dict(
        type = 'log',
        title = 'Revenue'),
        )

In [50]:
intensity_data

41       52.699935
53       43.017489
161       1.843845
317      45.888835
629     123.271937
641     369.733006
785      23.129968
857      41.231184
881      90.304108
953      81.743232
977      66.114719
1049     78.607846
Name: intensity, dtype: float64

In [51]:
trace1 = go.Scatter(x=x_median_y, 
                    y=y_median_y, 
                    showlegend = False, 
                    name='Median y',
                    mode = "lines",
                    line = dict(color='gray', 
                                width=2, 
                                dash='dash'))

trace2 = go.Scatter(x=x_median_x, 
                         y=y_median_x, 
                         showlegend = False, 
                         name='Median x', 
                         mode = "lines",
                         line = {'color':'gray', 
                                 'width':2, 
                                 'dash':'dash'}
                        )

trace3 = go.Scatter(
    x=merged_df_sector['scope1_plus_scope2'], 
    y=merged_df_sector['Revenue_num_tradingview'],
    showlegend = False,
    name="benchmark", 
    text=merged_df_sector['company_name'],
    mode = 'markers+text',
    line = {'color':'black'}, 
    marker = dict (
        color=intensity_data,
        size=25,
        showscale=True,
        colorscale='Hot_r',
        line=dict(
                color='black',
                width=1,
            )

        ),
    hovertemplate = '%{text}<br><b>Revenue:</b>%{y}'+ '<br><b>Emissions:</b> %{x:.2s}', # + '<br><b>Intensity: </b> %{intensity_data}',
    textposition='top center'
    )

fig = go.Figure(data = [trace1, trace2, trace3], 
                layout = layout)
                
fig.update_layout(
    autosize=False,
    width=1000,
    height=500,
    margin=dict(
        l=50,
        r=50,
        b=50,
        t=80,
        pad=4
    ),
    paper_bgcolor="#eeebf0",
    plot_bgcolor = '#eeebf0',
)
config = {'displaylogo': False}
fig.add_trace(
    go.Scatter(
        mode='markers',
        x=[corp_x],
        y=[corp_y],
        text = [corp_name],
        marker=dict(
            color='rgba(255,0,0, 0.01)',#'rgba(135, 206, 250, 0.01)',
            size=25,
            line=dict(
                color='red',
                width=3,
            )
        ),
        
        hovertemplate = '%{text}<br><b>Revenue:</b>%{y}'+ '<br><b>Emissions:</b> %{x:.2s}', # + '<br><b>Intensity: </b> %{intensity_data}',
        showlegend=False
    )
)

In [52]:
offline.iplot(fig, config = config)

In [56]:
    path = Path(os.path.dirname (os.getcwd()))
    XLSX_PATH = os.path.join(path.parent.parent, 'sp100.xlsx')
    
    COLS_TO_USE = {
    'companies':['company_id', 'company_name'],
    'ghg_quant':['company_id', 'ghg_scope_1','ghg_loc_scope_2','ghg_mkt_scope_2', 'reporting_year', "Source"],
    'grouping':['company_id', 'Sector1'],
    'financials':['company_id', 'Revenue_num_tradingview']
    }

    all_df = []
    for sheetname, cols in COLS_TO_USE.items():
        all_df.append(pd.read_excel(
            XLSX_PATH, 
            sheet_name = sheetname,
            engine = 'openpyxl', 
            usecols = cols
            ))
    merged_df = all_df[0]
    for i in range(len(COLS_TO_USE) - 1):
        merged_df = pd.merge(
            left = merged_df,
            right = all_df[i+1],
            how="left",
            on="company_id"
        )
    # Select Final Figures and Year 2019
    cond1 = (merged_df['reporting_year'] == 2019)
    cond2 = (merged_df['Source']== 'Final')
    filter_cond = cond1 & cond2
    merged_df = merged_df.loc[filter_cond]

    a =  merged_df[merged_df['company_id']==company_id]['Sector1']
    if len(a) > 0:
        sector = a.iloc[0]
    merged_df_sector = merged_df.loc[merged_df['Sector1']==sector]

    scope1 = pd.to_numeric(merged_df_sector['ghg_scope_1'], errors = 'coerce')
    scope2_loc = pd.to_numeric(merged_df_sector['ghg_loc_scope_2'], errors = 'coerce')
    scope2_mkt = pd.to_numeric(merged_df_sector['ghg_mkt_scope_2'], errors = 'coerce')

    merged_df_sector = merged_df_sector.assign(scope1_plus_scope2_loc= scope1 + scope2_loc)
    merged_df_sector = merged_df_sector.assign(scope1_plus_scope2_mkt= scope1 + scope2_mkt)
    merged_df_sector['scope1_plus_scope2'] = merged_df_sector[['scope1_plus_scope2_loc','scope1_plus_scope2_mkt']].min(axis=1)

    fields = ['company_id','company_name','ghg_scope_1', 'Sector1','Revenue_num_tradingview','scope1_plus_scope2']
    merged_df_sector = merged_df_sector [fields]

    merged_df_sector = merged_df_sector.dropna()
    merged_df_sector = merged_df_sector.assign(intensity=merged_df_sector['scope1_plus_scope2'] / merged_df_sector['Revenue_num_tradingview'])
    corp_x = merged_df_sector[merged_df_sector['company_id']==company_id]['scope1_plus_scope2'].iloc[0]
    corp_y = merged_df_sector[merged_df_sector['company_id']==company_id]['Revenue_num_tradingview'].iloc[0]
    corp_name = merged_df_sector[merged_df_sector['company_id']==company_id]['company_name'].iloc[0]
    x0 = merged_df_sector['scope1_plus_scope2']
    y0 = merged_df_sector['Revenue_num_tradingview']
    intensity_data = 10000000 * merged_df_sector['intensity']
    #index_corp = merged_df_sector.loc[merged_df_sector['company_id'] == company_id].index
    #merged_df_sector.drop(index_corp , inplace=True)
    x_median_x = [x0.min(), x0.max()]
    y_median_x = [y0.median(), y0.median()]
    x_median_y = [x0.median(), x0.median()]
    y_median_y = [y0.min(), y0.max()]

    layout = go.Layout (
        title = '<b>Operational Emissions Intensity Benchmark</b><br>Sector: ' + sector,
        title_x = 0.5,
        titlefont = dict(
            family = 'Arial',
            size = 16),
        plot_bgcolor = 'antiquewhite',
        xaxis =  dict(
            autorange = "reversed",
            type = 'log',
            title = '<i>high</i>----------<b>Scope1+2 Emissions</b>----------<i>low</i>'),
        yaxis = dict(
            type = 'log',
            title = 'Revenue'),
            )

    trace1 = go.Scatter(x=x_median_y, 
                        y=y_median_y, 
                        showlegend = False, 
                        name='Median y',
                        mode = "lines",
                        line = dict(color='gray', 
                                    width=2, 
                                    dash='dash'))

    trace2 = go.Scatter(x=x_median_x, 
                            y=y_median_x, 
                            showlegend = False, 
                            name='Median x', 
                            mode = "lines",
                            line = {'color':'gray', 
                                    'width':2, 
                                    'dash':'dash'}
                            )

    trace3 = go.Scatter(
        x=merged_df_sector['scope1_plus_scope2'], 
        y=merged_df_sector['Revenue_num_tradingview'],
        showlegend = False,
        name="benchmark", 
        text=merged_df_sector['company_name'],
        mode = 'markers+text',
        line = {'color':'black'}, 
        marker = dict (
            color=intensity_data,
            size=25,
            showscale=True,
            colorscale='Hot_r',
            line=dict(
                    color='black',
                    width=1,
                )

            ),
        hovertemplate = '%{text}<br><b>Revenue:</b>%{y}'+ '<br><b>Emissions:</b> %{x:.2s}', # + '<br><b>Intensity: </b> %{intensity_data}',
        textposition='top center'
        )

    fig = go.Figure(data = [trace1, trace2, trace3], 
                    layout = layout)
                    
    
    fig.add_trace(
        go.Scatter(
            mode='markers',
            x=[corp_x],
            y=[corp_y],
            text = [corp_name],
            marker=dict(
                color='rgba(255,0,0, 0.01)',#'rgba(135, 206, 250, 0.01)',
                size=25,
                line=dict(
                    color='red',
                    width=3,
                )
            ),
            
            hovertemplate = '%{text}<br><b>Revenue:</b>%{y}'+ '<br><b>Emissions:</b> %{x:.2s}', # + '<br><b>Intensity: </b> %{intensity_data}',
            showlegend=False
        )
    )

    
    full_fig = fig.full_figure_for_development(warn=False)

    x_range = full_fig.layout.xaxis.range
    y_range = full_fig.layout.yaxis.range
                    
    fig.update_layout(
        autosize=False,
        width=1000,
        height=500,
        margin=dict(
            l=50,
            r=50,
            b=50,
            t=80,
            pad=4
        ),
        paper_bgcolor="#eeebf0",
        plot_bgcolor = '#eeebf0',
    )